In [1]:
import sys
sys.path.insert(0, '../src/')

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import scipy.sparse as sp
from scipy.sparse import load_npz

import torch

import utils
from net import start_experiments

In [2]:
_A_obs = load_npz('../data/datasets/CORA_ML.npz')

In [3]:
val_share = 0.1
test_share = 0.05
seed = 481516234

train_ones, val_ones, val_zeros, test_ones, test_zeros = utils.train_val_test_split_adjacency(_A_obs, val_share, test_share, seed, undirected=True, connected=True, asserts=False)

train_graph = sp.csr_matrix((np.ones(len(train_ones)),(train_ones[:,0], train_ones[:,1])))
assert (train_graph.toarray() == train_graph.toarray().T).all()

In [7]:
models = start_experiments(num_experiments=5,
                           experiment_root='../logs/CORA-ML/Ours_test_loss',
                           train_graph=train_graph,
                           H=12,
                           optimizer=torch.optim.Adam,
                           optimizer_args={'lr': 0.1,
                                           'weight_decay': 1e-7},
                           invoke_every=2,
                           steps = 30,
                           val_edges=(val_ones, val_zeros))


Experiment_0
Step:  2/30, Loss: 7.92112, Edge-Overlap: 0.002
Step:  4/30, Loss: 7.64706, Edge-Overlap: 0.003
Step:  6/30, Loss: 7.08384, Edge-Overlap: 0.007
Step:  8/30, Loss: 6.27957, Edge-Overlap: 0.025
Step: 10/30, Loss: 5.42838, Edge-Overlap: 0.089
Step: 12/30, Loss: 4.74669, Edge-Overlap: 0.206
Step: 14/30, Loss: 4.28199, Edge-Overlap: 0.307
Step: 16/30, Loss: 3.93177, Edge-Overlap: 0.375
Step: 18/30, Loss: 3.64574, Edge-Overlap: 0.420
Step: 20/30, Loss: 3.42129, Edge-Overlap: 0.455
Step: 22/30, Loss: 3.24306, Edge-Overlap: 0.502
Step: 24/30, Loss: 3.09707, Edge-Overlap: 0.538
Step: 26/30, Loss: 2.97350, Edge-Overlap: 0.566
Step: 28/30, Loss: 2.87230, Edge-Overlap: 0.595
Step: 30/30, Loss: 2.78647, Edge-Overlap: 0.617

Experiment_1
Step:  2/30, Loss: 7.92331, Edge-Overlap: 0.001
Step:  4/30, Loss: 7.65387, Edge-Overlap: 0.002
Step:  6/30, Loss: 7.09832, Edge-Overlap: 0.008
Step:  8/30, Loss: 6.29560, Edge-Overlap: 0.023
Step: 10/30, Loss: 5.43182, Edge-Overlap: 0.090
Step: 12/30,